In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import glob
import lightgbm as lgb
from tqdm import tqdm
from loaders.blog_loader import load_data, mono_list
from monotonenorm import SigmaNet, GroupSort, direct_norm

In [2]:
Xtr, Ytr, Xts, Yts = load_data(get_categorical_info=False)

In [3]:
clf = lgb.LGBMRegressor(n_estimators=10000, max_depth=5, learning_rate=.1)
clf.fit(Xtr, Ytr, early_stopping_rounds=200, eval_set=[(Xts, Yts)], eval_metric='rmse', verbose=0)

LGBMRegressor(max_depth=5, n_estimators=10000)

In [4]:
rmse_tr = (((clf.predict(Xtr) - Ytr)**2).mean())**.5
rmse_ts = (((clf.predict(Xts) - Yts)**2).mean())**.5
rmse_tr, rmse_ts

(0.14392944329222293, 0.14531402077325561)

In [45]:

per_layer_lip = 10
width = 128

direct_norm = lambda x, *args, **kwargs: x # make it a normal network

model = torch.nn.Sequential(
  direct_norm(torch.nn.Linear(Xtr.shape[1], width), kind="one", alpha=per_layer_lip),
  GroupSort(width//2),
  direct_norm(torch.nn.Linear(width, width), kind="one", alpha=per_layer_lip),
  GroupSort(width//2),
  direct_norm(torch.nn.Linear(width, width), kind="one", alpha=per_layer_lip),
  GroupSort(width//2),
  direct_norm(torch.nn.Linear(width, 1), kind="one", alpha=per_layer_lip),
).cuda()
#model = SigmaNet(model, sigma=per_layer_lip**4, monotone_constraints=[int(i in mono_list) for i in range(Xtr.shape[1])])

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-3)
print('params:', sum(p.numel() for p in model.parameters()))
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.9999)


params: 68609


In [49]:
Xtrt = torch.tensor(Xtr, dtype=torch.float32)
Ytrt = torch.tensor(Ytr, dtype=torch.float32).view(-1, 1)
Xtst = torch.tensor(Xts, dtype=torch.float32).cuda()
Ytst = torch.tensor(Yts, dtype=torch.float32).view(-1, 1).cuda()

dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Xtrt, Ytrt), batch_size=1024, shuffle=True, pin_memory=True)

bar = tqdm(range(1000))
for i in bar:
  for Xi, yi in dataloader:
    y_pred = model(Xi.cuda())
    losstr = torch.nn.functional.mse_loss(y_pred, yi.cuda())
    optimizer.zero_grad()
    losstr.backward()
    optimizer.step()
    scheduler.step()

  with torch.no_grad():
    y_predts = model(Xtst)
    lossts = torch.nn.functional.mse_loss(y_predts, Ytst)
    bar.set_description(f'Loss: {losstr.item():.4f} {lossts.item():.4f}, RMSE: {(losstr.item()**.5):.4f} {(lossts.item()**.5):.4f}')

  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)